# Blog post analysis

In [1]:
# import Bach
from bach_open_taxonomy import ObjectivFrame

In [2]:
# connect to SQL db
of = ObjectivFrame.from_objectiv_data(db_url='postgresql://postgres:@localhost:5432/objectiv',
                                      table_name='data',
                                      time_aggregation='YYYY-MM-DD',
                                      start_date='2022-02-02')

In [3]:
# adding specific contexts to the data
of['application'] = of.global_contexts.gc.get_from_context_with_type_series(type='ApplicationContext', key='id')
of['root_location'] = of.location_stack.ls.get_from_context_with_type_series(type='RootLocationContext', key='id')
of['feature_nice_name'] = of.location_stack.ls.nice_name

## We had 315 users on the site following the post

In [4]:
# model hub: unique users, total month
users = of.model_hub.aggregate.unique_users(time_aggregation='YYYY-MM')
users.sort_index(ascending=False).head(10)

moment
2022-02    315
Name: unique_users, dtype: int64

In [5]:
# model hub: unique users, daily
users = of.model_hub.aggregate.unique_users()
users.sort_index(ascending=False).head(10)

moment
2022-02-04     13
2022-02-03    100
2022-02-02    239
Name: unique_users, dtype: int64

## They spent on average 4.36 mins on site

In [6]:
# model hub: duration
duration = of.model_hub.aggregate.session_duration(time_aggregation='YYYY-MM')
duration.sort_index(ascending=False).head(10)

moment
2022-02   0 days 00:04:36.909768
Name: session_duration, dtype: timedelta64[ns]

## Most users on blog, then home, then closely docs

In [7]:
# users by RootLocation
users_root = of.groupby(['root_location']).agg({'user_id':'nunique'})
users_root.sort_values('user_id_nunique', ascending=False).head()

,user_id_nunique
root_location,
blog,182
home,126
docs,116
about,42
jobs,40


## Quantile

In [48]:
# duration for each session
session_durations = of.groupby(['session_id']).agg({'moment':['min','max']})
session_durations['duration'] = session_durations['moment_max'] - session_durations['moment_min']

# calculate quantiles
quantiles = [0.25, 0.5, 0.75]
session_durations['duration'].quantile(q=quantiles).head()

AttributeError: 'SeriesTimedelta' object has no attribute 'quantile'

## Users spent 7 mins(!) on home, 6.36 in docs 

In [18]:
of[(of.root_location == 'home')].model_hub.aggregate.session_duration(time_aggregation='YYYY-MM').head()

moment
2022-02   0 days 00:07:15.380532
Name: session_duration, dtype: timedelta64[ns]

In [17]:
of[(of.root_location == 'docs')].model_hub.aggregate.session_duration(time_aggregation='YYYY-MM').head()

moment
2022-02   0 days 00:06:36.085650
Name: session_duration, dtype: timedelta64[ns]

In [16]:
of[(of.root_location == 'blog')].model_hub.aggregate.session_duration(time_aggregation='YYYY-MM').head()

moment
2022-02   0 days 00:02:40.471593
Name: session_duration, dtype: timedelta64[ns]

## Video Bach did well in blog and on home (adding in blog was good)

In [8]:
# select PressEvent and MediaStart event, to focus on user actions
users_feature = of[(of.event_type == 'PressEvent') |
                   (of.event_type == 'MediaStartEvent')]

# users by feature
users_feature = users_feature.groupby(['feature_nice_name', 'event_type']).agg({'user_id':'nunique'})
users_feature.sort_values('user_id_nunique', ascending=False).head()

,,user_id_nunique
feature_nice_name,event_type,
Media Player: objectiv-in-2-minutes located at Root Location: blog => Content: post-meet-objectiv-open-source-product-analytics-designed-for-data-sc,MediaStartEvent,74
Media Player: 2-minute-video located at Root Location: home,MediaStartEvent,36
Link: logo located at Root Location: blog => Navigation: navbar-top,PressEvent,24
Pressable: hamburger located at Root Location: blog => Navigation: navbar-top,PressEvent,21
Link: about-us located at Root Location: home => Navigation: navbar-top,PressEvent,20


## Docs: Modeling, Tracking, Quickstart even

In [9]:
# users by feature
users_feature.sort_values('user_id_nunique', ascending=False).head(10)

,,user_id_nunique
feature_nice_name,event_type,
Media Player: objectiv-in-2-minutes located at Root Location: blog => Content: post-meet-objectiv-open-source-product-analytics-designed-for-data-sc,MediaStartEvent,74
Media Player: 2-minute-video located at Root Location: home,MediaStartEvent,36
Link: logo located at Root Location: blog => Navigation: navbar-top,PressEvent,24
Pressable: hamburger located at Root Location: blog => Navigation: navbar-top,PressEvent,21
Link: about-us located at Root Location: home => Navigation: navbar-top,PressEvent,20
Pressable: hamburger located at Root Location: home => Navigation: navbar-top,PressEvent,18
Link: docs located at Root Location: home => Navigation: navbar-top,PressEvent,13
Link: Modeling located at Root Location: docs => Navigation: navbar-top,PressEvent,12
Link: quickstart-guide located at Root Location: blog => Content: post-meet-objectiv-open-source-product-analytics-designed-for-data-sc => Content: blog-post-try-objectiv,PressEvent,11


## GitHub links small numbers, on blog & home did better than top nav

In [10]:
# select github LinkContext only
of['link_id'] = of.location_stack.ls.get_from_context_with_type_series(type='LinkContext', key='id')
github_users = of[(of.link_id == 'objectiv-on-github')|
                  (of.link_id == 'github')]

# github users by feature
github_users = github_users.groupby(['feature_nice_name']).agg({'user_id':'nunique'})
github_users.sort_values('user_id_nunique', ascending=False).head()

,user_id_nunique
feature_nice_name,
Link: objectiv-on-github located at Root Location: blog => Content: post-meet-objectiv-open-source-product-analytics-designed-for-data-sc => Content: blog-post-try-objectiv,9
Link: objectiv-on-github located at Root Location: home => Content: the-stack,9
Link: github located at Root Location: home => Navigation: navbar-top,3
Link: github located at Root Location: blog => Navigation: navbar-top => Overlay: hamburger-menu,2
Link: github located at Root Location: home => Navigation: navbar-top => Overlay: hamburger-menu,1


## What did users do most on home?

In [11]:
# select homepage RootLocation only
home_users = of[(of.root_location == 'home')]

# select PressEvent and MediaStart event, to focus on user actions
home_users = home_users[(home_users.event_type == 'PressEvent') |
                   (home_users.event_type == 'MediaStartEvent')]

# home users by feature
home_users = home_users.groupby(['feature_nice_name']).agg({'user_id':'nunique'})
home_users.sort_values('user_id_nunique', ascending=False).head(10)

,user_id_nunique
feature_nice_name,
Media Player: 2-minute-video located at Root Location: home,36
Link: about-us located at Root Location: home => Navigation: navbar-top,20
Pressable: hamburger located at Root Location: home => Navigation: navbar-top,18
Link: docs located at Root Location: home => Navigation: navbar-top,13
Link: blog located at Root Location: home => Navigation: navbar-top,10
Link: objectiv-on-github located at Root Location: home => Content: the-stack,9
Link: about-us located at Root Location: home => Navigation: navbar-top => Overlay: hamburger-menu,9
Link: logo located at Root Location: home => Navigation: navbar-top,8
Link: faq located at Root Location: home => Navigation: navbar-top,7
